# Code created for Report in following doc:
G:\SALES\2022 Sales Reports\Sell-Through Reporting\Amazon\Analysis

In [48]:
# Importing the appropriate libraries we'll need for this code

import pandas as pd
import numpy as np
import os
import datetime 
from datetime import timedelta,datetime as dt
import datetime as dt
import pyodbc
import glob
pd.set_option('display.max_columns', None)


In [49]:
# SQL python connection to our server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

In [50]:
# Figure out the dates for the weekly data
weeknum = int(input("What week number are we in? "))

What week number are we in? 46


# Creating dates for the first week of the year and the current rolling week in a string format for the sql query.

In [51]:
# Sets the week 1 dates in the Amazon files for each year set in python datetime format.
ty_wk1 = datetime.date(2022,1,8)
ly_wk1 = datetime.date(2021,1,9)
py_wk1 = datetime.date(2020,1,4)
ppy_wk1 = datetime.date(2019,1,12)

In [52]:
ty = ty_wk1.year

ty

2022

In [53]:
# This calculates the corresponding week date for each year
current_ty = ty_wk1 + timedelta(weeks = weeknum-1)
current_ly = ly_wk1 + timedelta(weeks = weeknum-1)
current_py = py_wk1 + timedelta(weeks = weeknum-1)
current_ppy = ppy_wk1 + timedelta(weeks = weeknum-1)

In [54]:
# Sets week 1 dates as a string format for the sql query
date_ty_wk1_str = ty_wk1.strftime("%Y-%m-%d")
date_ly_wk1_str = ly_wk1.strftime("%Y-%m-%d")
date_py_wk1_str = py_wk1.strftime("%Y-%m-%d")
date_ppy_wk1_str = ppy_wk1.strftime("%Y-%m-%d")

In [55]:
# Sets current week date as a string format for the sql query
date_ty_current_str = current_ty.strftime("%Y-%m-%d")
date_ly_current_str = current_ly.strftime("%Y-%m-%d")
date_py_current_str = current_py.strftime("%Y-%m-%d")
date_ppy_current_str = current_ppy.strftime("%Y-%m-%d")

# Creating dates 10 weeks past to properly calculate the weeks on hand

In [56]:
# Calculates 10 weeks into the past
date_current_ty_minus10 = current_ty + timedelta(weeks = -9)
date_current_ly_minus10 = current_ly + timedelta(weeks = -9)
date_current_py_minus10 = current_py + timedelta(weeks = -9)
date_current_ppy_minus10 = current_ppy + timedelta(weeks = -9)

In [57]:
date_current_ty_minus10_str = date_current_ty_minus10.strftime("%Y-%m-%d")
date_current_ly_minus10_str = date_current_ly_minus10.strftime("%Y-%m-%d")
date_current_py_minus10_str = date_current_py_minus10.strftime("%Y-%m-%d")
date_current_ppy_minus10_str = date_current_ppy_minus10.strftime("%Y-%m-%d")

# Creating the period strings for the first month and current month for the SQL query.

In [58]:
#Calculates the corresponding "period" for week1 of each year
period_ty_wk1 = ty_wk1.strftime("%Y%m")
period_ly_wk1 = ly_wk1.strftime("%Y%m")
period_py_wk1 = py_wk1.strftime("%Y%m")
period_ppy_wk1 = ppy_wk1.strftime("%Y%m")

#Calculates the corresponding "period" for current week of each year
period_ty_current = current_ty.strftime("%Y%m")
period_ly_current = current_ly.strftime("%Y%m")
period_py_current = current_py.strftime("%Y%m")
period_ppy_current = current_ppy.strftime("%Y%m")

#Calculates the corresponding "period" for the week 10 weeks prior to the current 
period_ty_minus10 = date_current_ty_minus10.strftime("%Y%m")
period_ly_minus10 = date_current_ly_minus10.strftime("%Y%m")
period_py_minus10 = date_current_py_minus10.strftime("%Y%m")
period_ppy_minus10 = date_current_ppy_minus10.strftime("%Y%m")

# List of dates and periods

In [59]:
# Dates
ls_date_wk1 = list([ty_wk1,ly_wk1,py_wk1,ppy_wk1])
ls_date_current_wk = list([current_ty,current_ly,current_py,current_ppy])
ls_date_wk1_str = list([date_ty_wk1_str,date_ly_wk1_str,date_py_wk1_str,date_ppy_wk1_str])
ls_date_current_str = list([date_ty_current_str,date_ly_current_str,date_py_current_str,date_ppy_current_str])
ls_date_current_minus10_str = list([date_current_ty_minus10_str,date_current_ly_minus10_str,date_current_py_minus10_str,date_current_ppy_minus10_str])

# Periods
ls_period_wk1 = list([period_ty_wk1,period_ly_wk1,period_py_wk1,period_ppy_wk1])
ls_period_current = list([period_ty_current,period_ly_current,period_py_current,period_ppy_current])
ls_period_minus10 = list([period_ty_minus10,period_ly_minus10,period_py_minus10,period_ppy_minus10])

# Basic
ls_years = list(['ty','ly','py','ppy'])

In [60]:
print(ls_date_current_str)
print(ls_date_current_minus10_str)
print(ls_period_minus10)

['2022-11-19', '2021-11-20', '2020-11-14', '2019-11-23']
['2022-09-17', '2021-09-18', '2020-09-12', '2019-09-21']
['202209', '202109', '202009', '201909']


# Sell-In Queries

In [61]:
# TY sellin query
query_sellin = f'''select
    t.ITEM_TITLE ISBN
    ,sum(sd.QUANTITY_INVOICED) qty
    ,sum(sd.REVENUE_AMOUNT) val
from
    ebs.sales as sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
    inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID= stie.SSRRowID  
    inner join ebs.Item as t on sd.ITEM_ID=t.ITEM_ID
where
    sd.PERIOD between ? and ?
    and sd.TRX_DATE <= ?
    and ssr_row.SSRRowID ='6'
    and t.PRODUCT_TYPE in ('bk', 'ft')
    and t.PUBLISHER_CODE not in('Benefit','AFO LLC','Glam Media','PQ Blackwell')
group by
    t.ITEM_TITLE
'''

In [62]:
# YTD Sell-Thru Data

ls_df_sellin = [pd.read_sql_query(query_sellin,conn,params = [w,c,d])\
                                 for w,c,d in zip(ls_period_wk1,ls_period_current,ls_date_current_str)]

   # Item table metadata

In [63]:
query_item = '''
SELECT
    i.PUBLISHER_CODE Publisher
    ,i.PRODUCT_TYPE pt
    ,i.REPORTING_CATEGORY cat
    ,i.PUBLISHING_GROUP pgrp
    ,i.ITEM_TITLE ISBN
    ,i.SHORT_TITLE title
    ,i.PRICE_AMOUNT price
    ,i.AMORTIZATION_DATE pub
    ,case
        when i.AMORTIZATION_DATE is not null then year(i.AMORTIZATION_DATE)
        when substring(i.season,1,4) <> 'No S' then substring(i.season,1,4)
        else year(getdate())
    end [year]
FROM ebs.Item i
WHERE i.PRODUCT_TYPE in ('bk', 'ft')
'''

In [64]:
df_item = pd.read_sql_query(query_item,conn)

# Query for OnHand & CustomerOrders from the Sellthrough

In [65]:
query_onhand_cust = '''
SELECT
    st.ISBN
    ,sum(CustomerOrders) [CustomerOrders]
    ,sum(OnHand) [OnHand]
FROM
    [CBQ2].[cb].[Sellthrough_Amazon] st
        inner join ebs.Item i on i.ISBN = st.ISBN
WHERE
    st.[week] = ?
    AND i.PRODUCT_TYPE in ('bk', 'ft')
GROUP BY
    st.ISBN
'''

In [66]:
ls_df_onhand = [pd.read_sql_query(query_onhand_cust,conn,params = [x]) for x in ls_date_current_str]

In [67]:
ls_date_current_str

['2022-11-19', '2021-11-20', '2020-11-14', '2019-11-23']

In [68]:
ls_date_current_str

['2022-11-19', '2021-11-20', '2020-11-14', '2019-11-23']

# Sell-thru Queries and dataframes

In [69]:
query_sellthru = '''
SELECT
    st.[ISBN]
    ,sum([UnitShipped]) [ytd]
FROM
    [CBQ2].[cb].[Sellthrough_Amazon] st
        inner join ebs.item i on i.ISBN = (case
                                            when len(st.ISBN)=12 then '0' + st.ISBN
                                            when len(st.ISBN)=11 then '00' + st.ISBN
                                            else st.ISBN end)
WHERE
    st.[week] between ? and ?
    AND i.PRODUCT_TYPE in ('bk', 'ft')
GROUP BY
    st.[ISBN]
'''

In [70]:
ls_df_sellthru = [pd.read_sql_query(query_sellthru,conn,params = [w,c])
                  for w,c in zip(ls_date_wk1_str,ls_date_current_str)]

In [71]:
print(len(ls_df_sellthru))

for w,c in zip(ls_date_wk1_str,ls_date_current_str):
    print(w,c)

4
2022-01-08 2022-11-19
2021-01-09 2021-11-20
2020-01-04 2020-11-14
2019-01-12 2019-11-23


# Sell-thru Query between current and 10 weeks ago

In [72]:
# 10 Week Grouping
for w,c in zip(ls_date_current_minus10_str,ls_date_current_str):
    print(w,c)

2022-09-17 2022-11-19
2021-09-18 2021-11-20
2020-09-12 2020-11-14
2019-09-21 2019-11-23


In [73]:
# Sell-Thru 10 weeks
ls_df_10wks_sellthru = [pd.read_sql_query(query_sellthru,conn,params = [w,c])
                  for w,c in zip(ls_date_current_minus10_str,ls_date_current_str)]

In [74]:
for df in ls_df_10wks_sellthru:
    df.columns = ['ISBN','wks10']

# Add leading zeroes to the sellthru/onhand ISBN columns with less than 13 characters

In [75]:
# This function adds a zero before isbn's that have less than 13 characters
def isbn_leadzero(df):
    df['ISBN'] = df['ISBN'].apply(lambda x: '{0:0>13}'.format(x))
    return df

In [76]:
ls_df_sellin = [isbn_leadzero(df) for df in ls_df_sellin]
ls_df_sellthru = [isbn_leadzero(df) for df in ls_df_sellthru]
ls_df_onhand = [isbn_leadzero(df) for df in ls_df_onhand]
ls_df_sellthru_minus10 = [isbn_leadzero(df) for df in ls_df_10wks_sellthru]

In [77]:
ls_df_onhand[0]

,ISBN,CustomerOrders,OnHand
0,5055923712566,8.0,4
1,5055923712603,28.0,47
2,5055923712610,17.0,60
3,5055923747087,13.0,3
4,5055923751909,4.0,3
...,...,...,...
8905,9791036313578,0.0,8
8906,9791036313592,45.0,12
8907,9791036314940,0.0,5
8908,9791036339172,3.0,39


# Creating yearly tables

In [78]:
# This creates a loop that goes through the 3 sets of lists (sellin,sellthru, onhand)
# and creates yearly dataframes and puts them in this list below called, "df_yearly"

df_yearly = list()

for sellin, sellthru,minus,onhand in zip(ls_df_sellin,ls_df_sellthru,ls_df_sellthru_minus10,ls_df_onhand):
    df_year = pd.merge(sellin,sellthru,how = 'outer',on = 'ISBN').fillna(0)
    df_year = pd.merge(df_year,minus.loc[:,['ISBN','wks10']],how = 'outer',on='ISBN').fillna(0)
    df_year = pd.merge(df_year,onhand.loc[:,['ISBN','OnHand']],how = 'outer',on='ISBN').fillna(0)
    
    filt1 = (df_year.qty==0) & (df_year.val==0) & (df_year.ytd==0) & (df_year.OnHand==0)
    df_year = df_year.loc[~filt1]
    
    column_names = {'qty':'sellin_qty','val':'sellin_val','ytd':'sellthru_qty_ytd','wks10':'last10wks','OnHand':'sellthru_OnHand'}
    df_year.rename(columns = column_names,inplace=True)
    
    df_year = pd.merge(df_year,df_item,how = 'left',on='ISBN')
    
    # Note that these titles are not in the ebs.item list are mysteriously show up in our feed.
    filt2 = df_year.Publisher.isnull()
    df_year = df_year.loc[~filt2]
    
    df_year = df_year.loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title','price',\
                         'pub','year', 'sellin_qty','sellin_val', 'sellthru_qty_ytd','last10wks', 'sellthru_OnHand']]
    
    df_yearly.append(df_year)

# Creating unique isbn list for combo page

In [79]:
df_combo_isbns = pd.concat(df_yearly)
df_combo_isbns = df_combo_isbns.loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title','price','pub']]
df_combo_isbns.drop_duplicates(inplace = True)

In [80]:
# Creating the first Sellin Units Section along with the variance Section
df_combo_si = df_combo_isbns.copy()

for df in ls_df_sellin:
    df_combo_si = pd.merge(df_combo_si,df.loc[:,['ISBN','qty']],how = 'left',on = 'ISBN').fillna(0)

df_combo_si.columns = ['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si', 'py_si', 'ppy_si']

df_combo_si['ty_ly_si'] = ((df_combo_si['ty_si']-df_combo_si['ly_si'])/df_combo_si['ly_si']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_si['ty_py_si'] = ((df_combo_si['ty_si']-df_combo_si['py_si'])/df_combo_si['py_si']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_si['ty_ppy_si'] = ((df_combo_si['ty_si']-df_combo_si['ppy_si'])/df_combo_si['ppy_si']).replace([np.inf,-np.inf], np.nan).fillna(0)

In [81]:
# Creating the first Sellthru Units Section along with the variance Section
df_combo_st = df_combo_isbns.copy()

for df in ls_df_sellthru:
    df_combo_st = pd.merge(df_combo_st,df.loc[:,['ISBN','ytd']],how = 'left',on = 'ISBN').fillna(0)

df_combo_st.columns = ['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_st', 'ly_st', 'py_st', 'ppy_st']

df_combo_st['ty_ly_st'] = ((df_combo_st['ty_st']-df_combo_st['ly_st'])/df_combo_st['ly_st']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_st['ty_py_st'] = ((df_combo_st['ty_st']-df_combo_st['py_st'])/df_combo_st['py_st']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_st['ty_ppy_st'] = ((df_combo_st['ty_st']-df_combo_st['ppy_st'])/df_combo_st['ppy_st']).replace([np.inf,-np.inf], np.nan).fillna(0)

In [82]:
# Creating the first OnHand Units Section along with the variance Section
df_combo_oh = df_combo_isbns.copy()

for df in ls_df_onhand:
    df_combo_oh = pd.merge(df_combo_oh,df.loc[:,['ISBN','OnHand']],how = 'left',on = 'ISBN').fillna(0)

df_combo_oh.columns = ['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_oh', 'ly_oh', 'py_oh', 'ppy_oh']

df_combo_oh['ty_ly_oh'] = ((df_combo_oh['ty_oh']-df_combo_oh['ly_oh'])/df_combo_oh['ly_oh']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_oh['ty_py_oh'] = ((df_combo_oh['ty_oh']-df_combo_oh['py_oh'])/df_combo_oh['py_oh']).replace([np.inf,-np.inf], np.nan).fillna(0)
df_combo_oh['ty_ppy_oh'] = ((df_combo_oh['ty_oh']-df_combo_oh['ppy_oh'])/df_combo_oh['ppy_oh']).replace([np.inf,-np.inf], np.nan).fillna(0)

In [83]:
# The goal is to join the 3 sections above to create the combo tab
df_combo_tab = pd.merge(df_combo_si,df_combo_st,how='left',\
                        on=['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub'])

df_combo_tab = pd.merge(df_combo_tab,df_combo_oh,how='left',\
                        on=['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub'])

In [84]:
# Sorting the combo tab so it's descending by Sell in
df_combo_tab.sort_values('ty_si',ascending=False,inplace=True)

# Top Deltas Tab

In [85]:
# Creating the copy of the combo tab and looking at the variances between this year and last year - sellin - sellthru
df_combo_tab_var = df_combo_tab.copy()

df_combo_tab_var['TY_vs_LY_Sellin'] = df_combo_tab_var['ty_si'] - df_combo_tab_var['ly_si']
df_combo_tab_var['TY_vs_LY_SellThru'] = df_combo_tab_var['ty_st'] - df_combo_tab_var['ly_st']

In [86]:
#Check a row of numbers
print(df_combo_tab_var[df_combo_tab_var['ISBN']=='9781452179612'])
print(df_combo_tab_var.columns)

     Publisher  pt cat pgrp           ISBN                  title  price  \
457  Chronicle  BK  FO  FWN  9781452179612  From Crook to Cook hc  24.95   

                     pub     ty_si    ly_si    py_si   ppy_si  ty_ly_si  \
457  2018-10-01 00:00:00  122396.0  95217.0  96975.0  35735.0  0.285443   

     ty_py_si  ty_ppy_si     ty_st    ly_st    py_st   ppy_st  ty_ly_st  \
457   0.26214   2.425101  115176.0  91694.0  65303.0  23565.0  0.256091   

     ty_py_st  ty_ppy_st   ty_oh    ly_oh    py_oh  ppy_oh  ty_ly_oh  \
457  0.763717   3.887588  8158.0  23134.0  12485.0  8441.0 -0.647359   

     ty_py_oh  ty_ppy_oh  TY_vs_LY_Sellin  TY_vs_LY_SellThru  
457 -0.346576  -0.033527          27179.0            23482.0  
Index(['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si', 'py_si', 'ppy_si', 'ty_ly_si', 'ty_py_si',
       'ty_ppy_si', 'ty_st', 'ly_st', 'py_st', 'ppy_st', 'ty_ly_st',
       'ty_py_st', 'ty_ppy_st', 'ty_oh', 'ly_oh', 'py_oh', 'ppy

In [87]:
# Adding the year to this df so I can filter below
df_combo_isbns_yr = pd.concat(df_yearly)
df_combo_isbns_yr = df_combo_isbns_yr.loc[:,['ISBN','year']]
df_combo_isbns_yr.drop_duplicates(inplace = True)

In [88]:
df_combo_tab_var = pd.merge(df_combo_tab_var,df_combo_isbns_yr,on='ISBN')

In [89]:
## Sell Thru
# Creating the top 12 CHL variances
filt_chl = (df_combo_tab_var['pgrp'] == 'CHL') & (df_combo_tab_var['year'] != 2022)

df_chl_21_22_st = df_combo_tab_var.loc[filt_chl].sort_values('TY_vs_LY_SellThru')\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_st', 'ly_st','TY_vs_LY_SellThru']]
df_chl_22_21_st = df_combo_tab_var.loc[filt_chl].sort_values('TY_vs_LY_SellThru',ascending=False)\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_st', 'ly_st','TY_vs_LY_SellThru']]

# Creating the top 12 LIF variances
filt_lif = (df_combo_tab_var['pgrp'] == 'LIF') & (df_combo_tab_var['year'] != 2022)

df_lif_21_22_st = df_combo_tab_var.loc[filt_lif].sort_values('TY_vs_LY_SellThru')\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_st', 'ly_st','TY_vs_LY_SellThru']]
df_lif_22_21_st = df_combo_tab_var.loc[filt_lif].sort_values('TY_vs_LY_SellThru',ascending=False)\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_st', 'ly_st','TY_vs_LY_SellThru']]

In [90]:
df_combo_tab_var.columns

Index(['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si', 'py_si', 'ppy_si', 'ty_ly_si', 'ty_py_si',
       'ty_ppy_si', 'ty_st', 'ly_st', 'py_st', 'ppy_st', 'ty_ly_st',
       'ty_py_st', 'ty_ppy_st', 'ty_oh', 'ly_oh', 'py_oh', 'ppy_oh',
       'ty_ly_oh', 'ty_py_oh', 'ty_ppy_oh', 'TY_vs_LY_Sellin',
       'TY_vs_LY_SellThru', 'year'],
      dtype='object')

In [91]:
## Sell In
# Creating the top 12 CHL variances
filt_chl = (df_combo_tab_var['pgrp'] == 'CHL') & (df_combo_tab_var['year'] != ty_wk1.year)

df_chl_21_22_si = df_combo_tab_var.loc[filt_chl].sort_values('TY_vs_LY_Sellin')\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si','TY_vs_LY_Sellin']]
df_chl_22_21_si = df_combo_tab_var.loc[filt_chl].sort_values('TY_vs_LY_Sellin',ascending=False)\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si','TY_vs_LY_Sellin']]

# Creating the top 12 LIF variances
filt_lif = (df_combo_tab_var['pgrp'] == 'LIF') & (df_combo_tab_var['year'] != ty_wk1.year)

df_lif_21_22_si = df_combo_tab_var.loc[filt_lif].sort_values('TY_vs_LY_Sellin')\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si','TY_vs_LY_Sellin']]
df_lif_22_21_si = df_combo_tab_var.loc[filt_lif].sort_values('TY_vs_LY_Sellin',ascending=False)\
    .head(12).loc[:,['Publisher', 'pt', 'cat', 'pgrp', 'ISBN', 'title', 'price', 'pub',
       'ty_si', 'ly_si','TY_vs_LY_Sellin']]

In [92]:
ls_deltas = [df_chl_22_21_st,df_chl_21_22_st,df_lif_22_21_st,df_lif_21_22_st,df_chl_22_21_si,df_chl_21_22_si,df_lif_22_21_si,df_lif_21_22_si]

def add_row(df):
    df['row_num'] = np.arange(len(df))+1
    return df

for ls in ls_deltas:
    add_row(ls)

In [93]:
df_top_delta_st = pd.concat([df_chl_22_21_st,df_chl_21_22_st,df_lif_22_21_st,df_lif_21_22_st])
df_top_delta_si = pd.concat([df_chl_22_21_si,df_chl_21_22_si,df_lif_22_21_si,df_lif_21_22_si])

# Save off to Excel

In [94]:
path = r'G:\SALES\2022 Sales Reports\Sell-Through Reporting\Amazon\Analysis\py_drop\amaz_sellin_thru_py.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

for df,y in zip(df_yearly,ls_years):
    df.to_excel(writer,sheet_name=y,index = False)

df_combo_tab.to_excel(writer,sheet_name="Combo_tab",index = False)
df_top_delta_st.to_excel(writer,sheet_name="Delta_st",index = False)
df_top_delta_si.to_excel(writer,sheet_name="Delta_si",index = False)

writer.save()